### Happiness Analysis and rank countries  

- General exploratory analysis on top and lowest ranking countries for each measure 
- Find top 20 happiest countries and the top 20 least happiest countries and use this as ranking countries 
- Drop data irrelevant to analysis

In [ ]:
CREATE TABLE Happiness (
	countryname VARCHAR(100),
	year INT,
	lifeladder FLOAT,
	gdp FLOAT,
	socialsupport FLOAT,
	life_expectancy_birth FLOAT,
	perception_corruption FLOAT)

DELETE FROM happiness
WHERE year < 2012

-- top/lowest 20 happiest countries avg score between 2012-2022

SELECT DISTINCT countryname,
    ROUND(AVG(lifeladder)::numeric, 2) AS happiness_avg
FROM happiness
GROUP BY countryname
ORDER BY happiness_avg DESC
LIMIT 20;

--looking at top/lowest social support scores 

SELECT DISTINCT countryname,
    ROUND(AVG(socialsupport)::numeric, 2) AS support_avg
FROM happiness
GROUP BY countryname
ORDER BY support_avg DESC
LIMIT 10;

-- create table looking at only rank countries (top20/lowest20 countries) adding countryid

CREATE TABLE Country(
	countryid SERIAL PRIMARY KEY NOT NULL, 
	countryname VARCHAR(100) NOT NULL,
	avg_happiness_score FLOAT)

INSERT INTO happiness_rank(countryname, avg_happiness_score)
SELECT countryname,
       ROUND(AVG(lifeladder)::numeric, 2) AS happiness_avg
FROM happiness
GROUP BY countryname
ORDER BY happiness_avg DESC
LIMIT 20;

--happiness score for rank countries looking at every year 2012-2022 (for correlation analysis)

CREATE TABLE happiness_rank_years AS
(SELECT r.countryid, h.countryname, h.year, h.lifeladder
FROM happiness h 
RIGHT JOIN happiness_rank r 
ON r.countryname = h.countryname
ORDER BY r.countryid);

### Correlation Analysis 

#### Category: Health 

In [ ]:
CREATE TABLE health
(countryname VARCHAR(100),
seriesname VARCHAR (255),
"2012" FLOAT,
"2013" FLOAT,
"2014" FLOAT,
"2015" FLOAT,
"2016" FLOAT,
"2017" FLOAT,
"2018" FLOAT,
"2019" FLOAT,
"2020" FLOAT,
"2021" FLOAT,
"2022" FLOAT)

CREATE TABLE uhc
(countryname VARCHAR(100),
seriesname VARCHAR (255),
"2012" FLOAT,
"2013" FLOAT,
"2014" FLOAT,
"2015" FLOAT,
"2016" FLOAT,
"2017" FLOAT,
"2018" FLOAT,
"2019" FLOAT,
"2020" FLOAT,
"2021" FLOAT,
"2022" FLOAT)

CREATE TABLE health_uhc AS
(SELECT *
FROM health
UNION
SELECT *
FROM uhc
ORDER BY countryname, seriesname)

DROP TABLE health

DROP TABLE uhc

ALTER TABLE health_uhc
RENAME TO health

-- transpose wide table to long 
-- transpose to long table for easier analysis and visualization

CREATE TABLE health_long AS
(SELECT countryname, seriesname, '2012' AS year, "2012" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2013' AS year, "2013" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2014' AS year, "2014" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2015' AS year, "2015" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2016' AS year, "2016" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2017' AS year, "2017" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2018' AS year, "2018" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2019' AS year, "2019" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2020' AS year, "2020" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2021' AS year, "2021" AS value FROM health 
UNION ALL
SELECT countryname, seriesname, '2022' AS year, "2022" AS value FROM health) 

DROP TABLE health

-- add countryid 

CREATE TABLE health AS (
SELECT c.countryid, c.countryname, h.seriesname, h.year, h.value
FROM public."Country" c
LEFT JOIN health_long h ON
c.countryname = h.countryname 
GROUP BY c.countryid, c.countryname, h.seriesname, h.year, h.value
ORDER BY c.countryid, h.year)

-- join happiness score with health indicators (corelational analysis)
--convert year text to integer for join with rank countries and happiness scores 

ALTER TABLE health
ALTER COLUMN year TYPE integer USING year::integer

CREATE TABLE health_happiness_rank AS (
SELECT h.countryid, h.countryname, h.year, h.lifeladder, he.seriesname, he.value
FROM happiness_rank_years h
LEFT JOIN health he ON
h.countryid = he.countryid AND h.countryname = he.countryname AND h.year = he.year
GROUP BY  h.countryid, h.countryname, h.year, h.lifeladder, he.seriesname, he.value
ORDER BY h.countryid, h.countryname, h.year)

-- subquery: transpose series name into its own columns 
-- correlation: compare each seriesname dataset to happiness score 

SELECT 
    corr("lifeladder", "health_gdp") AS corr_gdp,
    corr("lifeladder", "health_capita") AS corr_capita,
	corr("lifeladder", "life_expectancy") AS corr_life_expectancy,
	corr("lifeladder", "mortality") AS corr_mortality,
	corr("lifeladder", "safe_water") AS corr_safewater,
	corr("lifeladder", "undernourishment") AS corr_undernourish,
    corr("lifeladder", "uhc") AS corr_uhc
FROM (
    SELECT 
        countryid, 
        year, 
        lifeladder,
        MAX(CASE WHEN seriesname = 'Domestic general government health expenditure (% of GDP)' THEN value END) AS "health_gdp",
        MAX(CASE WHEN seriesname = 'Domestic general government health expenditure per capita (current US$)' THEN value END) AS "health_capita",
        MAX(CASE WHEN seriesname = 'Life expectancy at birth, total (years)' THEN value END) AS "life_expectancy",
		MAX(CASE WHEN seriesname = 'Mortality rate, infant (per 1,000 live births)' THEN value END) AS "mortality",
		MAX(CASE WHEN seriesname = 'People using safely managed drinking water services (% of population)' THEN value END) AS "safe_water",
		MAX(CASE WHEN seriesname = 'Prevalence of undernourishment (% of population)' THEN value END) AS "undernourishment",
		MAX(CASE WHEN seriesname = 'UHC service coverage index' THEN value END) AS "uhc"
    FROM 
        health_happiness_rank
    GROUP BY 
        countryid, 
        countryname, 
        year, 
        lifeladder
) AS subquery
WHERE 
     "health_gdp" IS NOT NULL 
    AND "health_capita" IS NOT NULL
	AND "life_expectancy" IS NOT NULL
	AND "mortality" IS NOT NULL
	AND "safe_water" IS NOT NULL
	AND "undernourishment" IS NOT NULL 
    AND "uhc" IS NOT NULL "

#### Government 
- continue query structure for each category
- here is an example when one category had multiple different datasets with different measures 

In [ ]:
CREATE TABLE government
(countryname VARCHAR(100),
seriesname VARCHAR (255),
"2012" FLOAT,
"2013" FLOAT,
"2014" FLOAT,
"2015" FLOAT,
"2016" FLOAT,
"2017" FLOAT,
"2018" FLOAT,
"2019" FLOAT,
"2020" FLOAT,
"2021" FLOAT,
"2022" FLOAT)

-- two tables that are under the government category to combine together

CREATE TABLE government2 
(countryname VARCHAR(100),
seriesname VARCHAR (255),
"2012" FLOAT,
"2013" FLOAT,
"2014" FLOAT,
"2015" FLOAT,
"2016" FLOAT,
"2017" FLOAT,
"2018" FLOAT,
"2019" FLOAT,
"2020" FLOAT,
"2021" FLOAT,
"2022" FLOAT)


CREATE TABLE government_government2 AS
(SELECT *
FROM government 
UNION
SELECT *
FROM government2
ORDER BY countryname, seriesname)

DROP TABLE government

DROP TABLE government2

ALTER TABLE government_political
RENAME TO government 

--transpose wide table to long 

--transpose to long table for easier analysis and visualization

CREATE TABLE government_long AS
SELECT countryname, seriesname, '2012' AS year, "2012" AS value FROM government 
UNION ALL
SELECT countryname, seriesname, '2013' AS year, "2013" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2014' AS year, "2014" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2015' AS year, "2015" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2016' AS year, "2016" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2017' AS year, "2017" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2018' AS year, "2018" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2019' AS year, "2019" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2020' AS year, "2020" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2021' AS year, "2021" AS value FROM government
UNION ALL
SELECT countryname, seriesname, '2022' AS year, "2022" AS value FROM government

DROP TABLE government 

-- add countryid 

CREATE TABLE government AS (
SELECT c.countryid, c.countryname, g.seriesname, g.year, g.value
FROM public."Country" c
LEFT JOIN government_long g ON
c.countryname = g.countryname 
GROUP BY c.countryid, c.countryname, g.seriesname, g.year, g.value
ORDER BY c.countryid, g.year)

--- add another data set violent crime rates (already in long format)

CREATE TABLE original_crime(
    country_name VARCHAR(255),
    series_name VARCHAR(255),
    year INTEGER,
    unit_of_measurement VARCHAR(100),
    value BIGINT)

SELECT *
FROM original_crime
WHERE country_name = 'Canada' AND
year = '2014' AND 
unit_of_measurement <> 'Counts' AND 
series_name = 'Violent offences'

--- we'll only look at rate of violent offences because there are multiple types of violent offence values 
--- drop data that we cannot compare to our happiness rank (ex. before 2012)

DELETE FROM original_crime
WHERE year < 2012

DELETE FROM original_crime
WHERE unit_of_measurement = 'Counts'

ALTER TABLE original_crime
DROP COLUMN unit_of_measurement 

DELETE FROM original_crime 
WHERE series_name <> 'Violent offences'

-- take the median of violent offences for each year to get one value for each year
-- there are multiple values for violent offences per year because this dataset accounts for different types of violent offences 
--ex. kidnapping, sexual offences etc.
-- extract the median due to skewness of different violent offences 

CREATE TABLE crime_median AS (SELECT 
    country_name,
    series_name,
	year,
    percentile_cont(0.5) WITHIN GROUP (ORDER BY value) AS value 
FROM 
    original_crime
GROUP BY 
    country_name, series_name, year 
ORDER BY 
    country_name, series_name)

SELECT *
FROM crime_median

DROP TABLE crime 

--- extract data only from our happiness rank countries 

CREATE TABLE crime AS (SELECT
	c.countryid, c.countryname, cr.series_name, cr.year, cr.value
FROM public."Country" c
LEFT JOIN crime_median cr ON
c.countryname = cr.country_name
ORDER BY countryid, countryname, year)

SELECT *
FROM crime
WHERE value IS NULL 
ORDER BY countryid 

-- important to note 14 of the 20 ranked lowest happiness countries do not report their crime statistics (null)
-- correlation analysis happiness score and crime rate median 
--join happiness score with governemnt/political indicators (corelational analysis)
--convert year text to integer for join with rank countries and happiness scores 

ALTER TABLE government 
ALTER COLUMN year TYPE integer USING year::integer

CREATE TABLE g_happiness_rank AS (
SELECT h.countryid, h.countryname, h.year, h.lifeladder, g.seriesname, g.value
FROM happiness_rank_years h
LEFT JOIN government g ON
h.countryid = g.countryid AND h.countryname = g.countryname AND h.year = g.year
GROUP BY  h.countryid, h.countryname, h.year, h.lifeladder, g.seriesname, g.value
ORDER BY h.countryid, h.countryname, h.year)

--subquery: transpose series name into its own columns 
--correlation: compare each seriesname dataset to happiness score 

SELECT 
    corr("lifeladder", "control_corruption") AS corr_corruption,
    corr("lifeladder", "gov_effect") AS corr_government,
    corr("lifeladder", "political_stability") AS corr_stability,
    corr("lifeladder", "violence_rate") AS corr_violence
FROM (
    SELECT 
        countryid, 
        countryname, 
        year, 
        lifeladder,
        MAX(CASE WHEN seriesname = 'Control of Corruption: Percentile Rank' THEN value END) AS "control_corruption",
        MAX(CASE WHEN seriesname = 'Government Effectiveness: Percentile Rank' THEN value END) AS "gov_effect",
        MAX(CASE WHEN seriesname = 'Political Stability and Absence of Violence/Terrorism: Percentile Rank' THEN value END) AS "political_stability",
        MAX(CASE WHEN seriesname = 'Violent offences' THEN value END) AS "violence_rate"
    FROM 
        g_happiness_rank
    GROUP BY 
        countryid, 
        countryname, 
        year, 
        lifeladder
) AS subquery
WHERE 
    "control_corruption" IS NOT NULL 
    AND "gov_effect" IS NOT NULL 
    AND "political_stability" IS NOT NULL
    AND "violence_rate" IS NOT NULL

- continue correlational analysis for all categories (environment, education, economy) and visualize in Tableau 

### Regressional Analysis 
- Regression coefficent to see which inidcators have highest predictive impact on happiness score 
- Export aggregate data comparing happiness scores with data 
- Regressional analysis with pandas and statsmodels with python for statistical analysis 

In [ ]:
CREATE TABLE econ_happiness AS (
    SELECT 
        countryid, 
        countryname, 
        year, 
        lifeladder,
        MAX(CASE WHEN seriesname = 'Consumer price index (2010 = 100)' THEN value END) AS "cpi",
        MAX(CASE WHEN seriesname = 'GDP per capita (current US$)' THEN value END) AS "gdp_capita",
        MAX(CASE WHEN seriesname = 'GDP per capita, PPP (current international $)' THEN value END) AS "gdp_capita_ppp",
        MAX(CASE WHEN seriesname = 'Gini index' THEN value END) AS "gini",
		MAX(CASE WHEN seriesname = 'Poverty headcount ratio at national poverty lines (% of population)' THEN value END) AS "pov_pop",
        MAX(CASE WHEN seriesname = 'Unemployment, total (% of total labor force) (national estimate)' THEN value END) AS "unemployment"
    FROM 
        econ_happiness_rank
    GROUP BY 
        countryid, 
        countryname, 
        year, 
        lifeladder)

In [ ]:
import pandas as pd
import statsmodels.api as sm

data = pd.read_csv('econ_happiness.csv') 

--removing null values 
data = data.dropna() 

-- variables 
X = data[['cpi', 'gdp_capita', 'gdp_capita_ppp', 'gini', 'pov_pop', 'unemployment']]
y = data['lifeladder']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

- continue regressional analysis for each category
- explore datasets that are statistically significant (p value < 0.05, high coefficent) and which categories have highest R squared value
- export aggregate datasets with high regressional significance for visualization

### Average Happiness Scores and Indicator Scores for ALL countries
- combine data across all datasets for each country from 2012-2022 for map visualization 

In [ ]:
ALTER TABLE happiness
DROP COLUMN perception_corruption 

CREATE TABLE happiness_average AS 
(SELECT countryname, ROUND(AVG(lifeladder)::decimal, 2) AS happiness_score 
FROM 
happiness
GROUP BY 
countryname
ORDER BY 
countryname)

DROP TABLE happiness 

ALTER TABLE happiness_average 
RENAME TO happiness 

CREATE TABLE health_average AS (SELECT 
    countryname,
    MAX(CASE WHEN seriesname = 'Domestic general government health expenditure (% of GDP)' THEN average END) AS health_expenditure_gdp,
    MAX(CASE WHEN seriesname = 'Domestic general government health expenditure per capita (current US$)' THEN average END) AS health_expenditure_per_capita,
    MAX(CASE WHEN seriesname = 'Life expectancy at birth, total (years)' THEN average END) AS life_expectancy,
    MAX(CASE WHEN seriesname = 'Mortality rate, infant (per 1,000 live births)' THEN average END) AS infant_mortality_rate,
    MAX(CASE WHEN seriesname = 'People using at least basic drinking water services (% of population)' THEN average END) AS drinking_water_services,
    MAX(CASE WHEN seriesname = 'Prevalence of undernourishment (% of population)' THEN average END) AS undernourishment,
    MAX(CASE WHEN seriesname = 'UHC service coverage index' THEN average END) AS uhc_service_coverage
FROM 
    health_long
GROUP BY 
    countryname
ORDER BY 
    countryname)

-- continue for each category and each dataset to combine all tables for averages across all countries 

CREATE TABLE country_indicator_averages AS (SELECT 
    h.countryname,
    h.happiness_score,
    he.health_expenditure_gdp,
    he.health_expenditure_per_capita,
    he.life_expectancy,
    he.infant_mortality_rate, 
    he.drinking_water_services,
    he.undernourishment,
    he.uhc_service_coverage,
    ec.consumer_price_index,
    ec.gdp_per_capita,
    ec.gdp_per_capita_ppp,
    ec.gini_index,
    ec.poverty_headcount_ratio,
    ec.unemployment_rate,
    e.educational_attainment,
    e.government_expenditure_gdp,
    e.government_expenditure_total,
    e.literacy_rate,
    e.lower_secondary_completion_rate,
    ev.co2,
    ev.pm2,
    ev.protected_areas,
	g.control_of_corruption,
	g.government_effectiveness,
	g.political_stability,
    g.violence_rate
FROM 
    happiness h 
LEFT JOIN 
    health_average he ON h.countryname = he.countryname
LEFT JOIN 
    economy_average ec ON h.countryname = ec.countryname
LEFT JOIN 
    education_average e ON h.countryname = e.countryname
LEFT JOIN 
    enviroment_average ev ON h.countryname = ev.countryname
LEFT JOIN
	government_average g ON h.countryname = g.countryname 
ORDER BY 
    h.countryname)